<a href="https://colab.research.google.com/github/jackieshen2013/nyu-python-public-policy/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Statement: When you read through the descriptor column in the 311 data, you will see that complaints related to graffiti are actually scattered throughout multiple complaint_type categories. We want to identify all complaints related to graffiti and see which community districts have the most instances of graffiti.

To help make this assignment easier, I have created a smaller subset of the 311 data for you to use. It's named cleaned_311_data_hw3.csv and it's shared with you on Google Drive. This smaller dataset only contains ~65,000 records from relevant complaint type categories. df = pd.read_csv('/content/drive/My Drive/Data for Python/cleaned_311_data_hw3.csv', header='infer')

Step 1. Create a function that checks each row in the 311 dataframe to see if the word "graffiti" is present in either the complaint_type value or descriptor value. Both columns may contain the word, so you should check both. If the word "graffiti" is found, the function should return the boolean value True. If "graffiti" is not found, the function should return the boolean value False.
Hint 1: The same way that we checked if the string "BRONX" existed in the community_board value in the cb_counts table during the lecture, you can check if "graffiti" exists in the descriptor or complaint_type values.
Hint 2: The descriptor column contains some null/NaN values. You'll have trouble running a function on these. Before your function checks if "graffiti" is present in descriptor, the function should make sure that a descriptor value exists using pd.notnull(row.descriptor)
Hint 3: Capitalization may be inconsistent. It could help to use .lower() to convert strings to lowercase.
Step 2. Apply the function created in Step 1 to the 311 dataframe and create a new column called graffiti_flag that captures the output from the function.
Tip: There are two checks you can use to confirm that the function worked as expected.
Group by graffiti_flag to make sure there are records tagged as True.
Using only the records where graffiti_flag is True (sample[sample.grafitti_flag]), group by complaint_type to make sure that more than one complaint_type is included.
Step 3. Create another dataframe df_graffiti that only contains records where graffiti_flag is True. df_graffiti = df[df.grafitti_flag]
Step 4. Group your dataframe df_graffiti to get the count of requests per community_board. Use .nlargest() to identify which Community District has the highest count.

In [0]:
import pandas as pd
from google.colab import drive

In [0]:
import matplotlib.pyplot as plt 
import numpy as np

In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('/content/drive/My Drive/Data for Python/cleaned_311_data_hw3.csv', header='infer')

In [0]:
print(df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
df.head()

,Unnamed: 0,Unnamed: 0.1,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11434.0,NaN,NaN,NaN,NaN,BAISLEY BOULEVARD,MERRICK BOULEVARD,INTERSECTION,JAMAICA,NaN,NaN,Closed,08/04/2018 03:20:07 AM,The Service Request submitted did not have suf...,08/23/2018 11:15:00 AM,12 QUEENS,NaN,QUEENS,1048111.0,188479.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)"
2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,11206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,Closed,08/04/2018 04:19:15 AM,The Department of Parks and Recreation has com...,08/02/2018 08:19:55 AM,01 BROOKLYN,NaN,BROOKLYN,NaN,NaN,MOBILE,Ten Eyck Plaza,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAR ROCKAWAY,NaN,NaN,Closed,08/04/2018 07:32:33 AM,The Department of Parks and Recreation has com...,08/16/2018 10:28:38 AM,14 QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Westbourne Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLUSHING,NaN,NaN,Closed,08/04/2018 07:33:12 AM,The condition was inspected and it was determi...,09/04/2018 05:38:02 PM,07 QUEENS,NaN,QUEENS,NaN,NaN,MOBILE,Maple Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
set(df.community_board)

{'0 Unspecified',
 '01 BRONX',
 '01 BROOKLYN',
 '01 MANHATTAN',
 '01 QUEENS',
 '01 STATEN ISLAND',
 '02 BRONX',
 '02 BROOKLYN',
 '02 MANHATTAN',
 '02 QUEENS',
 '02 STATEN ISLAND',
 '03 BRONX',
 '03 BROOKLYN',
 '03 MANHATTAN',
 '03 QUEENS',
 '03 STATEN ISLAND',
 '04 BRONX',
 '04 BROOKLYN',
 '04 MANHATTAN',
 '04 QUEENS',
 '05 BRONX',
 '05 BROOKLYN',
 '05 MANHATTAN',
 '05 QUEENS',
 '06 BRONX',
 '06 BROOKLYN',
 '06 MANHATTAN',
 '06 QUEENS',
 '07 BRONX',
 '07 BROOKLYN',
 '07 MANHATTAN',
 '07 QUEENS',
 '08 BRONX',
 '08 BROOKLYN',
 '08 MANHATTAN',
 '08 QUEENS',
 '09 BRONX',
 '09 BROOKLYN',
 '09 MANHATTAN',
 '09 QUEENS',
 '10 BRONX',
 '10 BROOKLYN',
 '10 MANHATTAN',
 '10 QUEENS',
 '11 BRONX',
 '11 BROOKLYN',
 '11 MANHATTAN',
 '11 QUEENS',
 '12 BRONX',
 '12 BROOKLYN',
 '12 MANHATTAN',
 '12 QUEENS',
 '13 BROOKLYN',
 '13 QUEENS',
 '14 BROOKLYN',
 '14 QUEENS',
 '15 BROOKLYN',
 '16 BROOKLYN',
 '17 BROOKLYN',
 '18 BROOKLYN',
 '26 BRONX',
 '27 BRONX',
 '28 BRONX',
 '55 BROOKLYN',
 '56 BROOKLYN',
 '64

In [0]:
set(df.complaint_type)

{'Bridge Condition',
 'Broken Parking Meter',
 'Bus Stop Shelter Complaint',
 'Graffiti',
 'Highway Condition',
 'Maintenance or Facility',
 'Public Payphone Complaint',
 'Public Toilet',
 'School Maintenance'}

In [0]:
set(df.descriptor)

{'Air Conditioning Problem',
 'American Flag',
 'Asbestos',
 'Bent/Loose',
 'Broken Fence',
 'Broken Glass',
 'Broken Lock',
 'Broken Water Fountain',
 'Broken Window',
 'Cave-in',
 'Coin or Card Did Not Register',
 'Concrete Barrier',
 'Crash Cushion Defect',
 'Credit Card Stuck in Meter',
 'Damaged Bench',
 'Damaged Leg or Pole Bent',
 'Damaged Other',
 'Damaged Telephone',
 'Damaged Toilet/Sink',
 'Damaged or Leaking Roof',
 'Damaged or Missing Ad Box',
 'Dead Animal',
 'Decorative Necklace Lighting',
 'Dirt, Debris, Litter Complaint',
 'Dirty/Graffiti',
 'Fallen Debris from Bridge',
 'Fence',
 'Flooded',
 'Garbage or Litter',
 'Graffiti',
 'Graffiti - Bridge',
 'Graffiti - Highway',
 'Graffiti or Vandalism',
 'Graffiti/Litter on Phone',
 'Grass/Weeds',
 'Guard Rail - Bridge',
 'Guard Rail - Highway',
 'Heating Problem',
 'High Grass',
 'Highway Fence',
 'Hours of Operation',
 'Lack of Supplies',
 'Leaky Roof',
 'Lighting',
 'Litter',
 'Loose Plate',
 'Lost Coin',
 'Missing/Stump',


In [0]:
#1
def graffiti_check(row):
  if 'graffiti' in row.complaint_type.lower():
    return True 
  if pd.notnull(row.descriptor):
     if 'graffiti' in row.descriptor.lower():
         return True
  return False

In [0]:

type(graffiti_check)

function

In [75]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'unique_key', 'created_date',
       'closed_date', 'agency', 'agency_name', 'complaint_type', 'descriptor',
       'location_type', 'incident_zip', 'incident_address', 'street_name',
       'cross_street_1', 'cross_street_2', 'intersection_street_1',
       'intersection_street_2', 'address_type', 'city', 'landmark',
       'facility_type', 'status', 'due_date', 'resolution_description',
       'resolution_action_updated_date', 'community_board', 'bbl', 'borough',
       'x_coordinate_(state_plane)', 'y_coordinate_(state_plane)',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'vehicle_type', 'taxi_company_borough', 'taxi_pick_up_location',
       'bridge_highway_name', 'bridge_highway_direction', 'road_ramp',
       'bridge_highway_segment', 'latitude', 'longitude', 'location',
       'graffiti_flag'],
      dtype='object')

In [0]:
#2
df['graffiti_flag'] = df.apply(graffiti_check, axis=1)

In [0]:
#3
df_graffiti = df[['community_board']][df['graffiti_flag']]

In [72]:
df_graffiti.groupby('community_board').size().reset_index(name='count_of_grafitti_requests').nlargest(15, 'count_of_grafitti_requests')

,community_board,count_of_grafitti_requests
2,01 BROOKLYN,1797
13,03 MANHATTAN,1593
17,04 BROOKLYN,1429
12,03 BROOKLYN,1252
21,05 BROOKLYN,962
29,07 BROOKLYN,779
66,Unspecified BRONX,615
8,02 MANHATTAN,598
67,Unspecified BROOKLYN,591
4,01 QUEENS,572
